<a href="https://colab.research.google.com/github/Dulavinya/Tea_defect_detection/blob/main/Leaf_disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam

import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import requests
import csv

In [2]:
# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# Add custom layers for classification with dropout
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu",kernel_initializer = 'he_normal')(x)  # Reduced dense layer size
x = BatchNormalization()(x)

x = Dropout(0.5)(x)  # Increased dropout rate

output = Dense(7, activation="softmax")(x)

In [4]:
# Create the complete model
model = Model(inputs=base_model.input, outputs=output)

# Summary of the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         7,17

 Total params: 15,251,271 (58.18 MB)

 Trainable params: 15,249,223 (58.17 MB)

 Non-trainable params: 2,048 (8.00 KB)

In [5]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss="CategoricalCrossentropy", metrics=['accuracy'])

In [6]:
# Create an ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Rescale pixel values to [0, 1]
    rotation_range=30,  # Increased rotation range
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # Added shear range
    zoom_range=0.2,  # Added zoom range
    horizontal_flip=True,
    validation_split=0.2
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
zip_path = '/content/drive/MyDrive/diseases.zip'


!unzip -qo "{zip_path}" -d ./FER


In [9]:
!ls ./FER


teaLeafBD


In [10]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Training data generator with validation split
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training generator
train_generator = train_datagen.flow_from_directory(
    "/content/FER/teaLeafBD/teaLeafBD",
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training'   # important for splitting
)

# Validation generator
validation_generator = val_datagen.flow_from_directory(
    "/content/FER/teaLeafBD/teaLeafBD",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # important for splitting
)


Found 4223 images belonging to 7 classes.
Found 1053 images belonging to 7 classes.


In [11]:

# Get the mapping of class names to their corresponding numerical labels
class_indices = train_generator.class_indices

# Reverse the dictionary to get the mapping of labels to class names
classes = {v: k for k, v in class_indices.items()}

# Now, 'classes' contains the mapping from numerical labels to class names
print(classes)


label2id=dict(zip(classes.values(),classes.keys()))
print(label2id)

{0: '1. Tea algal leaf spot', 1: '2. Brown Blight', 2: '3. Gray Blight', 3: '4. Helopeltis', 4: '5. Red spider', 5: '6. Green mirid bug', 6: '7. Healthy leaf'}
{'1. Tea algal leaf spot': 0, '2. Brown Blight': 1, '3. Gray Blight': 2, '4. Helopeltis': 3, '5. Red spider': 4, '6. Green mirid bug': 5, '7. Healthy leaf': 6}


In [12]:
# Define the directory and filename for saving the best model
model_filename = 'leaf_deseas.keras'

# Combine the directory and filename
model_path = model_filename

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(model_path, save_best_only=True, monitor='val_loss', verbose=1)

In [13]:

best_weights = ModelCheckpoint('leaf_deseas.weights.h5',
                               monitor='val_loss',
                               verbose=0,
                               save_weights_only=True,
                               save_best_only=True,
                               mode='min',
                               save_freq='epoch')


last_weights = ModelCheckpoint('leaf_deseas.weights.h5',
                               monitor='val_loss',
                               verbose=0,
                               save_weights_only=True,
                               save_best_only=False,
                               mode='auto',
                               save_freq='epoch')

# Define EarlyStopping and ModelCheckpoint callbacks
early_stopping = EarlyStopping(monitor='val_loss',
                                       patience=25,
                                       restore_best_weights=True,
                                       verbose=1)

In [ ]:

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks = [checkpoint,best_weights,last_weights,early_stopping],
    verbose=1
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
